### **Disjoint-set/Union-Find**

La estructura de datos **Disjoint-Set**, también conocida como **Union-Find**, permite gestionar particiones de un conjunto de elementos de forma eficiente, ofreciendo operaciones de unión (`merge`) y búsqueda de representante (`find_partition`). Esta implementación en Python optimiza la complejidad amortizada mediante compresión de caminos y unión por rango, logrando costo casi constante en escenarios con numerosas operaciones. Es especialmente útil en algoritmos de grafos, detección de ciclos y construcciones de árboles de expansión mínima como Kruskal.



In [ ]:
"""
Implementación del TDA-Disjoint-Set / Union-Find con:

*  Compresión de caminos en find_partition
*  unión por rango en merge
*  Manejo exhaustivo de errores (mensajes en español)
*  Compatibilidad total con el juego de pruebas unitarias suministrado

"""
from collections.abc import Iterable, Mapping  # Mapping nos permite detectar dicts


#  Validación utilitaria 
def is_defined(val):
    return val is not None


def is_undefined(val):
    return val is None


def is_iterable(val):
    try:
        iter(val)
        return True
    except TypeError:
        return False


#  Mensajes de error 
def _error_union_find_constructor_illegal_argument(val):
    return f"Argumento ilegal para el constructor de DisjointSet: {val}"


def _error_union_find_constructor_duplicate_element(val):
    return (
        f"Elemento duplicado en el conjunto inicial para el constructor de DisjointSet: {val}"
    )


def _error_find_not_in_set(val):
    return f"El argumento {val} para el método find_partition no pertenece a este conjunto"


def _error_invalid_argument(method, param, val):
    return f"Argumento inválido para el método '{method}': el parámetro '{param}' tiene un valor inválido {val!r}"


# Clase auxiliar
class Info:
    """Mantiene la raíz y el rango de un elemento."""

    def __init__(self, elem):
        if not is_defined(elem):
            raise TypeError(_error_invalid_argument("Info.__init__", "elem", elem))
        self._root = elem
        self._rank = 1

    # root
    @property
    def root(self):
        return self._root

    @root.setter
    def root(self, new_root):
        if not is_defined(new_root):
            raise TypeError(
                _error_invalid_argument("Info.root.setter", "new_root", new_root)
            )
        self._root = new_root

    # rank
    @property
    def rank(self):
        return self._rank

    @rank.setter
    def rank(self, new_rank):
        self._rank = new_rank


#  Clase DisjointSet
class DisjointSet:
    """
    Estructura Conjuntos disjuntos (Union-Find) con:

    * find con compresión de caminos
    * unión por rango
    * interface amigable y validación estricta
    """

    # Constructor 
    def __init__(self, initial_set=None):
        # 0. Normalización
        if initial_set is None:
            initial_set = []

        # 1. Rechazar tipos explícitamente prohibidos:
        #    - str y bytes se considerarían colecciones de caracteres/bytes,
        #      pero el constructor debe tratarlos como argumento inválido.
        #    - Mapping (dict, OrderedDict, defaultdict, …) se itera sobre sus
        #      claves; tampoco se desea permitirlo.
        if isinstance(initial_set, (str, bytes)) or isinstance(initial_set, Mapping):
            raise TypeError(_error_union_find_constructor_illegal_argument(initial_set))

        # 2. Asegurar que sea iterable (list, tuple, set, generator, etc.)
        if not is_iterable(initial_set):
            raise TypeError(_error_union_find_constructor_illegal_argument(initial_set))

        # 3. Construir la tabla de elementos
        self._elements: dict = {}
        for elem in initial_set:
            if not is_defined(elem):
                raise TypeError(_error_union_find_constructor_illegal_argument(elem))
            if elem in self._elements:
                raise TypeError(_error_union_find_constructor_duplicate_element(elem))
            self._elements[elem] = Info(elem)

    # Propiedades de sólo lectura 
    @property
    def size(self):
        """Número total de elementos (no de subconjuntos) almacenados."""
        return len(self._elements)

    # Métodos públicos
    def add(self, elem):
        """
        Inserta `elem` como nuevo subconjunto singleton.

        Devuelve:
            True  - si el elemento se añadió.
            False - si ya existía previamente.
        """
        if not is_defined(elem):
            raise TypeError(_error_invalid_argument("add", "elem", elem))
        if elem in self._elements:
            return False
        self._elements[elem] = Info(elem)
        return True

    def find_partition(self, elem):
        """
        Devuelve el representante (raíz) del subconjunto que contiene a `elem`,
        aplicando compresión de caminos para minimizar la profundidad.
        """
        if is_undefined(elem):
            raise TypeError(_error_invalid_argument("find_partition", "elem", elem))
        if elem not in self._elements:
            raise TypeError(_error_find_not_in_set(elem))

        info = self._elements[elem]
        if info.root != elem:  # no es raíz ⇒ seguir recursivamente
            info.root = self.find_partition(info.root)
        return info.root

    def merge(self, elem1, elem2):
        """
        Une los subconjuntos que contienen elem1 y elem2.

        Retorna:
            True  - si los subconjuntos eran distintos y se fusionaron.
            False - si ambos elementos ya pertenecían al mismo subconjunto.
        """
        # Validación de argumentos se delega a find_partition
        r1 = self.find_partition(elem1)
        r2 = self.find_partition(elem2)
        if r1 == r2:
            return False  # ya estaban unidos

        info1 = self._elements[r1]
        info2 = self._elements[r2]

        # Unión por rango: el de mayor rango se convierte en la nueva raíz
        if info1.rank >= info2.rank:
            info2.root = info1.root
            info1.rank += info2.rank
        else:
            info1.root = info2.root
            info2.rank += info1.rank

        return True

    def are_disjoint(self, elem1, elem2):
        """
        Indica si elem1 y elem2 pertenecen a subconjuntos distintos.

        Devuelve True si son disjuntos, False! si comparten representante.
        """
        p1 = self.find_partition(elem1)
        p2 = self.find_partition(elem2)
        return p1 != p2


#### **Explicación del código**
El constructor de la clase `DisjointSet` acepta un iterable inicial de elementos y aplica validaciones estrictas para rechazar argumentos inválidos. Se prohíben tipos como `str`, `bytes` y `Mapping` (diccionarios), ya que su iteración original podría interpretarlos como colecciones de caracteres o claves. 

La función `is_iterable` verifica adaptativamente colecciones, mientras que `is_defined` e `is_undefined` controlan valores nulos. Cualquier error arroja mensajes detallados en español, por ejemplo, "Argumento ilegal para el constructor de DisjointSet" o "Elemento duplicado en el conjunto inicial", facilitando la depuración.

La clase auxiliar `Info` encapsula la información de cada elemento: su raíz (`root`) y su rango (`rank`). Mediante propiedades con getters y setters, garantiza la integridad de los datos, validando que la raíz nunca sea `None` y gestionando el incremento de rango sin restricciones implícitas. Este diseño modular desacopla la lógica de almacenamiento de metadatos de la funcionalidad principal, simplificando el mantenimiento y las extensiones posteriores.

El método `find_partition` aplica recursivamente compresión de caminos: comprueba si el elemento no es raíz y, en ese caso, reasigna su padre al representante final obtenido de la llamada recursiva. Este proceso minimiza la profundidad de los árboles resultantes, acelerando búsquedas subsecuentes. Además, controla argumentos indefinidos y valores no registrados, generando excepciones con mensajes claros como "El argumento X para el método find_partition no pertenece a este conjunto".

La función `merge` ejecuta unión por rango: compara los rangos de las raíces de dos elementos y asigna la raíz de mayor rango como representante común, sumando el rango del árbol absorbido a la raíz dominante. Si ambos subárboles ya comparten representante, retorna `False`; de lo contrario, efectúa la unión y devuelve `True`. El método `are_disjoint` complementa estas operaciones, indicando si dos elementos residen en subconjuntos diferentes. Finalmente, el método `add` incorpora nuevos elementos singulares con validación, asegurando unicidad y generando errores tipo `TypeError` para entradas inválidas.


Cómo usarlo: 



In [ ]:
ds = DisjointSet(['a', 'b', 'c'])
print(ds.size)                # 3
print(ds.are_disjoint('a','b'))  # True
ds.merge('a','b')
print(ds.are_disjoint('a','b'))  # False
ds.add('d')                   # True

#### **Pruebas unitarias**

Este archivo define una suite de pruebas unitarias con unittest para verificar el comportamiento de la clase `DisjointSet`.


In [ ]:
import unittest

# Si tu clase vive en otro archivo distinta a este, importa así:
# from disjointset import DisjointSet
#
# Como el enunciado indica que estamos en Jupyter ─o mismo espacio─
# se puede hacer:
from __main__ import DisjointSet   # noqa: E402


# 1. Constructor 
class TestConstructor(unittest.TestCase):

    def test_illegal_arg_not_iterable(self):
        with self.assertRaises(TypeError) as cm:
            DisjointSet('123')
        self.assertIn(
            "Argumento ilegal para el constructor de DisjointSet: 123",
            str(cm.exception)
        )

        with self.assertRaises(TypeError) as cm2:
            DisjointSet({'a': 1})
        self.assertIn(
            "Argumento ilegal para el constructor de DisjointSet: {'a': 1}",
            str(cm2.exception)
        )

    def test_illegal_arg_none_elem(self):
        with self.assertRaises(TypeError) as cm:
            DisjointSet([None])
        self.assertIn(
            "Argumento ilegal para el constructor de DisjointSet: None",
            str(cm.exception)
        )

    def test_duplicate_elements(self):
        with self.assertRaises(TypeError) as cm:
            DisjointSet(['1', '1'])
        self.assertIn(
            "Elemento duplicado en el conjunto inicial para el constructor de DisjointSet: 1",
            str(cm.exception)
        )

    def test_constructor_iterables_ok(self):
        DisjointSet([1, 2, 'a', (1, 2)])              # lista
        DisjointSet({'x', 'y'})                       # set
        DisjointSet({1: 'a', 2: 'b'}.items())         # dict.items()

    def test_constructor_different_but_alike(self):
        DisjointSet(['1', 1, '2', 2])
        DisjointSet({'1', 1, '2', 2})
        DisjointSet({('1', '2'), (1, 2)})


# 2. find_partition
class TestFindPartition(unittest.TestCase):

    def setUp(self):
        self.ds = DisjointSet(['1', '2', '3', 'a', 'abc'])

    def test_invalid_argument(self):
        with self.assertRaises(TypeError):
            self.ds.find_partition(None)

    def test_not_in_set(self):
        with self.assertRaises(TypeError):
            self.ds.find_partition('x')

    def test_initial_root_is_itself(self):
        for k in ['1', '2', '3', 'a', 'abc']:
            self.assertEqual(self.ds.find_partition(k), k)


# 3. merge
class TestMerge(unittest.TestCase):

    def setUp(self):
        self.ds = DisjointSet(['1', '2', '3', '4', '5', '6'])

    def test_invalid_arguments(self):
        with self.assertRaises(TypeError):
            self.ds.merge(None, None)

    def test_not_in_set(self):
        with self.assertRaises(TypeError):
            self.ds.merge('x', 'y')

    def test_basic_merge(self):
        self.assertTrue(self.ds.merge('1', '2'))
        self.assertEqual(
            self.ds.find_partition('1'),
            self.ds.find_partition('2')
        )

    def test_repeated_merge(self):
        self.ds.merge('5', '6')
        self.assertFalse(self.ds.merge('5', '6'))

    def test_union_by_rank(self):
        ds = DisjointSet(['1', '2', '3', '4', '5', '6'])
        self.assertTrue(ds.merge('1', '2'))
        root = ds.find_partition('1')

        ds.merge('1', '3')
        self.assertEqual(ds.find_partition('3'), root)

        ds.merge('3', '4')
        for x in ['1', '2', '3', '4']:
            self.assertEqual(ds.find_partition(x), root)

        ds.merge('5', '6')
        ds.merge('3', '6')
        for x in ['5', '6']:
            self.assertEqual(ds.find_partition(x), root)


# 4. are_disjoint 
class TestAreDisjoint(unittest.TestCase):

    def setUp(self):
        self.ds = DisjointSet(['1', '2', '3'])

    def test_invalid_arguments(self):
        with self.assertRaises(TypeError):
            self.ds.are_disjoint(None, None)
        with self.assertRaises(TypeError):
            self.ds.are_disjoint('x', 'y')

    def test_true_before_merge(self):
        self.assertTrue(self.ds.are_disjoint('1', '2'))

    def test_false_after_merge(self):
        self.ds.merge('3', '1')
        self.assertFalse(self.ds.are_disjoint('1', '3'))

    def test_commutativity(self):
        self.assertEqual(
            self.ds.are_disjoint('1', '3'),
            self.ds.are_disjoint('3', '1')
        )


# 5. add 
class TestAdd(unittest.TestCase):

    def setUp(self):
        self.ds = DisjointSet(['1', '2', '3', 1])

    def test_invalid_arguments(self):
        with self.assertRaises(TypeError):
            self.ds.add(None)

    def test_add_existing(self):
        for k in ['1', '2', '3', 1]:
            self.assertFalse(self.ds.add(k))

    def test_add_new(self):
        ds = DisjointSet(['1', '2', '3'])
        new_keys = ['new', (1, 2), frozenset({3}), 2]
        for k in new_keys:
            self.assertTrue(ds.add(k))
            self.assertEqual(ds.find_partition(k), k)


# 6. size
class TestSize(unittest.TestCase):

    def setUp(self):
        self.ds = DisjointSet(['1', '2', '3'])

    def test_size_consistent(self):
        self.assertEqual(self.ds.size, 3)

    def test_size_after_union(self):
        self.ds.merge('1', '2')
        self.assertEqual(self.ds.size, 3)
        self.ds.merge('3', '2')
        self.assertEqual(self.ds.size, 3)

    def test_size_after_add(self):
        old = self.ds.size
        self.assertTrue(self.ds.add('new'))
        self.assertEqual(self.ds.size, old + 1)
        self.assertFalse(self.ds.add('new'))
        self.assertEqual(self.ds.size, old + 1)


# Ejecución directa
if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)


#### **Implementación de Union-Find con listas de particiones**  

Esta versión de **union-find** mantiene un diccionario que mapea cada elemento a un `set` mutable que agrupa su partición.  

1. **Inicialización y validación**  
   - El constructor recibe un iterable y rechaza explícitamente `str`, `bytes`, `Mapping` y `None` para evitar ambigüedades.  
   - Cada elemento único se inicializa como un `set` singleton en `self._partitions`.  

2. **Operaciones básicas**  
   - `add(elem)`: añade un nuevo singleton si no existía, lanzando error en caso de `None`.  
   - `find_partition(elem)`: devuelve la referencia al `set` que contiene a `elem`, con validación de existencia.  
   - `merge(a, b)`: une dos particiones por **tamaño**, moviendo todos los elementos del conjunto más pequeño al más grande y actualizando sus referencias en O(min |A|, |B|).  
   - `are_disjoint(a, b)`: comprueba si `a` y `b` residen en distintos `set`.  

3. **API de conveniencia**  
   - `__len__`, `__contains__`, `__iter__` permiten usar `len(uf)`, `elem in uf` y bucles `for`.  
   - `groups()`: retorna la colección de particiones como `frozenset`, útil para inspección.

**Relación con algoritmos de grafos y clustering**  
- En **grafos**, Union-Find detecta componentes conectados: al iterar sobre aristas `(u, v)`, `merge(u, v)` agrupa nodos, y después `find_partition` identifica si dos vértices comparten componente, optimizando detección de ciclos y construcción de bosques de expansión mínima (Kruskal).  
- En **clustering aglomerativo**, cada elemento inicia en su propio clúster (`singleton`). Al aplicar un criterio de similitud, se va haciendo `merge` de los clústeres más cercanos.  Los `set` mutantes mantienen en tiempo amortizado la asignación de cada punto a su clúster global, facilitando consultas de pertenencia y actualización de grupos en cada paso de la jerarquía.

In [ ]:
"""
Versión mejorada (Python 3.10+) de una estructura Disjoint-Set / Unión-Búsqueda
implementada con listas de elementos agrupados en conjuntos.

- Validación exhaustiva y mensajes de error claros  
- Tipado estático con typing  
- Métodos utilitarios (__contains__, __len__, __iter__, groups)  
- Unión por tamaño O(min |A|, |B|)  
- Cobertura de casos extremos que suelen olvidarse (str, bytes, Mapping, None)
"""
from __future__ import annotations

from collections.abc import Iterable as ABCIterable, Mapping
from typing import Dict, Iterator, MutableSet, Set, TypeVar

T = TypeVar("T", bound=object)


# Utilidades
def _is_iterable(obj) -> bool:
    try:
        iter(obj)
        return True
    except TypeError:  # pragma: no cover  (solo entra si *no* es iterable)
        return False


# Errores
class UnionFindError(TypeError):
    """Señala un uso incorrecto de la API de Unión-Búsqueda."""


def _err_constructor_illegal(val):
    return f"Argumento ilegal para el constructor de UnionFindLists: {val!r}"


def _err_constructor_duplicate(val):
    return f"Elemento duplicado en el conjunto inicial del constructor de UnionFindLists: {val!r}"


def _err_find_not_in_set(val):
    return f"El argumento {val!r} para el método find_partition no está presente en esta estructura"


def _err_invalid_arg(method: str, param: str, val):
    return (
        f"Argumento inválido en '{method}': el parámetro '{param}' recibió el valor inválido {val!r}"
    )


# Clase principal
class UnionFindLists:
    """
    Unión-Búsqueda basado en conjuntos: cada partición se almacena como un conjunto mutable de sus elementos.

    A diferencia de las versiones "clásicas" basadas en árboles, esta implementación es más sencilla
    conceptualmente, pero merge cuesta O(min |A|, |B|) al trasladar todos los elementos del conjunto más pequeño al más grande.

    Parámetros
    ----------
    initial_set : Iterable[T] | None
        Conjunto inicial de elementos (cada uno arranca en su propia partición). Se rechazan
        explícitamente las cadenas de texto (str, bytes) y los mappings (dict, defaultdict, ...)
        para evitar confusiones.
    """

    # Constructor
    def __init__(self, initial_set: ABCIterable[T] | None = None) -> None:
        if initial_set is None:
            initial_set = []

        # 1. Filtro de tipos prohibidos (iterables "engañosos")
        if isinstance(initial_set, (str, bytes)) or isinstance(initial_set, Mapping):
            raise UnionFindError(_err_constructor_illegal(initial_set))

        # 2. Debe ser realmente iterable
        if not _is_iterable(initial_set):
            raise UnionFindError(_err_constructor_illegal(initial_set))

        # 3. Diccionario elemento → referencia al conjunto-partición
        self._partitions: Dict[T, MutableSet[T]] = {}

        # 4. Cargar elementos (verificando duplicados / None)
        for elem in initial_set:
            if elem is None:
                raise UnionFindError(_err_constructor_illegal(elem))
            if elem in self._partitions:
                raise UnionFindError(_err_constructor_duplicate(elem))
            self._partitions[elem] = {elem}

    # Propiedades de solo lectura
    def __len__(self) -> int:  # len(uf)
        return len(self._partitions)

    @property
    def size(self) -> int:  # alias explícito
        return len(self)

    # Iteración / consulta
    def __contains__(self, elem: T) -> bool:  # elem está en uf
        return elem in self._partitions

    def __iter__(self) -> Iterator[T]:  # para x en uf
        return iter(self._partitions)

    # Devuelve una 'vista' de las particiones (conjuntos únicos)
    def groups(self) -> Set[frozenset[T]]:
        """
        Conjunto de las particiones actuales como frozenset.
        Útil para depuración o para contar cuántos subconjuntos hay.
        """
        return {frozenset(g) for g in {id(s): s for s in self._partitions.values()}.values()}

    # Métodos principales
    def add(self, elem: T, /) -> bool:
        """Añade `elem` como partición de un solo elemento. True => insertado; False => ya existía."""
        if elem is None:
            raise UnionFindError(_err_invalid_arg("add", "elem", elem))
        if elem in self._partitions:
            return False
        self._partitions[elem] = {elem}
        return True

    def find_partition(self, elem: T, /) -> MutableSet[T]:
        """Devuelve un conjunto mutable que representa la partición de `elem`."""
        if elem is None:
            raise UnionFindError(_err_invalid_arg("find_partition", "elem", elem))
        try:
            return self._partitions[elem]
        except KeyError as exc:
            raise UnionFindError(_err_find_not_in_set(elem)) from exc

    def merge(self, elem1: T, elem2: T, /) -> bool:
        """
        Une (si es necesario) las particiones que contienen a `elem1` y `elem2`.

        Retorna
        -------
        True  – si las particiones eran distintas y se fusionaron.  
        False – si ambos elementos ya estaban en la misma partición.
        """
        p1 = self.find_partition(elem1)
        p2 = self.find_partition(elem2)
        if p1 is p2:  # ya unidas
            return False

        # Unión por tamaño: mover el conjunto más pequeño dentro del más grande
        if len(p1) < len(p2):
            small, big = p1, p2
        else:
            small, big = p2, p1

        big_update = big.update  # microoptimización
        for x in small:
            self._partitions[x] = big
        big_update(small)  # añade todos los elementos del conjunto pequeño

        return True

    def are_disjoint(self, elem1: T, elem2: T, /) -> bool:
        """Devuelve `True` si `elem1` y `elem2` están en particiones distintas."""
        return self.find_partition(elem1) is not self.find_partition(elem2)

    # Representación amigable
    def __repr__(self) -> str:
        groups = ", ".join(sorted(map(repr, self.groups())))
        return f"{self.__class__.__name__}({{{groups}}})"


In [ ]:
uf = UnionFindLists(['a', 'b', 'c'])
uf
uf.add('d')  
uf.add('a')

In [ ]:
uf.find_partition('b')

In [ ]:
"""
Implementación optimizada de Union-Find (bosque de conjuntos disjuntos) con:

- Compresión de caminos *in situ* (find => α(m, n))  
- Unión por rango (heurística de rango, O(1))  
- Tipado estático (Python 3.10+) y API "pythónica" (`len`, `in`, `iter`)  
- Mensajes de error claros mediante una excepción propia  
"""

from __future__ import annotations

from collections.abc import Iterable as ABCIterable, Mapping
from typing import Dict, Iterator, TypeVar

T = TypeVar("T", bound=object)


# Excepción propia
class UnionFindError(TypeError):
    """Uso indebido de la API Union-Find (argumentos nulos, inexistentes, etc.)."""


# Mensajes de error
def _err_ctor_illegal(val):
    return f"Argumento ilegal para el constructor de UnionFindTrees: {val!r}"


def _err_ctor_duplicate(val):
    return f"Elemento duplicado en el conjunto inicial para el constructor de UnionFindTrees: {val!r}"


def _err_find_not_in_set(val):
    return f"El argumento {val!r} para el método find_partition no está presente en esta estructura"


def _err_invalid_arg(method: str, param: str, val):
    return (
        f"Argumento inválido en '{method}': el parámetro '{param}' recibió el valor inválido {val!r}"
    )


# Clase principal
class UnionFindTrees:
    """
    Bosque de conjuntos disjuntos con punteros a padres, compresión de caminos y unión por rango.

    Parámetros
    ----------
    initial_set : Iterable[T] | None
        Elementos iniciales; cada uno comienza como singleton.
        Se rechazan explícitamente `str`, `bytes` y `Mapping`
        para evitar confusiones con iterables "engañosos".
    """

    __slots__ = ("_parent", "_rank")

    # Constructor
    def __init__(self, initial_set: ABCIterable[T] | None = None) -> None:
        if initial_set is None:
            initial_set = []

        # Rechazar textuales y mappings engañosos
        if isinstance(initial_set, (str, bytes)) or isinstance(initial_set, Mapping):
            raise UnionFindError(_err_ctor_illegal(initial_set))

        # Validar que sea iterable
        try:
            iterator = iter(initial_set)
        except TypeError:  # pragma: no cover
            raise UnionFindError(_err_ctor_illegal(initial_set)) from None

        # Inicializar estructuras
        self._parent: Dict[T, T] = {}
        self._rank: Dict[T, int] = {}

        # Cargar elementos de forma segura
        for elem in iterator:
            if elem is None:
                raise UnionFindError(_err_ctor_illegal(elem))
            if elem in self._parent:
                raise UnionFindError(_err_ctor_duplicate(elem))
            self._parent[elem] = elem  # se apunta a sí mismo
            self._rank[elem] = 1       # rango inicial

    # Metaprotocolos útiles
    def __len__(self) -> int:          # len(uf)
        return len(self._parent)

    def __contains__(self, elem: T) -> bool:  # elem in uf
        return elem in self._parent

    def __iter__(self) -> Iterator[T]:  # for x in uf
        return iter(self._parent)

    def __repr__(self) -> str:
        roots = {r for r in self._parent.values() if r == self._parent[r]}
        return f"{self.__class__.__name__}(n={len(self)}, conjuntos={len(roots)})"

    # Alias explícito
    @property
    def size(self) -> int:
        """Alias para obtener el tamaño (número de elementos)."""
        return len(self)

    # Función interna para validar elementos
    def _validate_elem(self, method: str, param: str, elem: T) -> None:
        if elem is None:
            raise UnionFindError(_err_invalid_arg(method, param, elem))
        if elem not in self._parent:
            raise UnionFindError(_err_find_not_in_set(elem))

    # API pública
    def add(self, elem: T, /) -> bool:
        """Inserta `elem` como singleton. True ⇒ añadido; False ⇒ ya existía."""
        if elem is None:
            raise UnionFindError(_err_invalid_arg("add", "elem", elem))
        if elem in self._parent:
            return False
        self._parent[elem] = elem
        self._rank[elem] = 1
        return True

    def find_partition(self, elem: T, /) -> T:
        """Devuelve la raíz (representante) de la partición que contiene `elem`."""
        self._validate_elem("find_partition", "elem", elem)

        # Compresión de caminos iterativa
        root = elem
        while root != self._parent[root]:
            root = self._parent[root]

        # Pasada para comprimir rutas
        while elem != root:
            parent = self._parent[elem]
            self._parent[elem] = root
            elem = parent

        return root

    def merge(self, elem1: T, elem2: T, /) -> bool:
        """
        Une las particiones que contienen `elem1` y `elem2`.

        Retorno
        -------
        True  - si las particiones eran distintas y se fusionaron.  
        False - si ya pertenecían a la misma partición.
        """
        r1 = self.find_partition(elem1)
        r2 = self.find_partition(elem2)
        if r1 == r2:
            return False

        # Unión por rango: el árbol de menor rango apunta al de mayor
        if self._rank[r1] < self._rank[r2]:
            r1, r2 = r2, r1

        self._parent[r2] = r1
        self._rank[r1] += self._rank[r2]
        return True

    def are_disjoint(self, elem1: T, elem2: T, /) -> bool:
        """True si `elem1` y `elem2` están en particiones distintas."""
        return self.find_partition(elem1) != self.find_partition(elem2)


Esta implementación de **Union-Find** (o Disjoint‐Set Forest) ofrece una estructura eficiente para gestionar particiones dinámicas de un conjunto de elementos mediante dos heurísticas clásicas: compresión de caminos e unión por rango.  

1. **Definición de la excepción y mensajes de error**  
   - Se crea una excepción propia `UnionFindError`, que hereda de `TypeError` para señalizar usos inválidos de la API (parámetros nulos, elementos ausentes, duplicados, tipos no permitidos).  
   - Cuatro funciones auxiliares (`_err_ctor_illegal`, `_err_ctor_duplicate`, `_err_find_not_in_set`, `_err_invalid_arg`) generan mensajes claros en tiempo de ejecución, usando `!r` para representar con comillas literales los valores problemáticos.

2. **Almacenamiento interno**  
   - Se emplean dos diccionarios:  
     - `_parent: Dict[T, T]`, que para cada elemento almacena su puntero al padre (o a sí mismo si es raíz).  
     - `_rank: Dict[T, int]`, que mantiene un valor heurístico de "rango" (aproximación al tamaño del subárbol) para decidir la raíz en las uniones.  
   - El constructor valida exhaustivamente el `initial_set`:  
     - Rechaza strings, bytes y cualquier `Mapping`.  
     - Asegura que el argumento sea iterable (captura `TypeError` al llamar `iter`).  
     - Parcela `None` y duplicados, lanzando `UnionFindError` con mensaje apropiado.  
     - Inicializa cada elemento como su propia raíz (`_parent[elem] = elem`) con rango 1.

3. **Metaprotocolos de Python**  
   - `__len__` devuelve el número total de elementos registrados.  
   - `__contains__` y `__iter__` permiten usar `elem in uf` y `for x in uf`.  
   - `__repr__` sintetiza el estado mostrando cuántos conjuntos hay (`sets=…`) y cuántos elementos (`n=…`).

4. **Validación interna de parámetros**  
   - `_validate_elem` centraliza la comprobación de que un elemento no sea `None` y exista en `_parent`, lanzando la excepción con mensaje personalizado.

5. **Operaciones clave**  
   - `add(elem)`: inserta un nuevo singleton, devolviendo `True` si fue añadido y `False` si ya existía. Valida `None`.  
   - `find_partition(elem)`:  
     - Primera fase: recorre punteros padre hasta hallar la raíz (condición `root != parent[root]`).  
     - Segunda fase: vuelve al elemento original y redirige cada puntero intermedio directamente a la raíz, comprimiendo caminos.  
     - Ambos bucles son lineales en la profundidad original, pero la compresión de caminos amortiza la complejidad a prácticamente constante inversa de la función de Ackermann, α(m,n).  
   - `merge(elem1, elem2)`:  
     - Obtiene las raíces `r1`, `r2` con `find_partition`.  
     - Si son iguales, retorna `False` (ya unidos).  
     - Si no, compara rangos y hace la raíz de mayor rango el padre de la otra, actualizando `_parent[r2] = r1` y sumando rangos: `_rank[r1] += _rank[r2]`.  
     - Esta heurística garantiza altura logarítmica en el peor caso y tiempo constante amortizado.  
   - `are_disjoint(a,b)`: compara raíces para saber si viven en conjuntos distintos.

Cada método arroja `UnionFindError` con mensajes claros cuando se violan las precondiciones, haciendo que la depuración sea sencilla y el uso de la estructura robusto en escenarios reales.


In [ ]:
uf = UnionFindTrees(['a', 'b', 'c'])
uf

In [ ]:
uf.add('d')      # nuevo ⇒ True
uf.add('a')      # 'a' ya existe ⇒ False
len(uf)          # alias de uf.size

In [ ]:
# a y b estaban en sets distintos
uf.merge('a', 'b')
uf.merge('a', 'b')   # ya unidas → False

# estado rápido
print(uf)     # __repr__ muestra elementos y nº de conjuntos


In [ ]:
uf.find_partition('a')      # raíz representante (compresión de caminos)
uf.are_disjoint('a', 'b') # 'a' y 'b' comparten raíz
uf.are_disjoint('a', 'c')      # aún separados


In [ ]:
'c' in uf
for x in uf:          # iterar elementos
    print(x, uf.find_partition(x))

**Uso en Kruskal’s Minimum Spanning Tree**  

Para construir un **árbol de expansión mínima** en un grafo no dirigido, el algoritmo de Kruskal sigue estos pasos:  

1. **Inicialización**: Crea un `UnionFindTrees` con todos los vértices del grafo. Cada vértice empieza en su propio conjunto.  
2. **Ordenación de aristas**: Lista todas las aristas `(u,v,w)` y ordenarlas por peso `w` ascendente.  
3. **Iteración sobre aristas**:  
   - Para cada arista `(u,v,w)` en orden, llama a `find_partition(u)` y `find_partition(v)`.  
   - Si las raíces difieren (uso de `are_disjoint(u,v)`), incluye la arista en el árbol resultante y ejecute `merge(u,v)`.  
   - Si las raíces coinciden, omite la arista para evitar ciclos.  
4. **Finalización**: Al procesar todas las aristas o alcanzar `n-1` aristas unidas, el conjunto de aristas acumuladas forma el MST.  

La eficiencia global es $O(E\log E)$ por la ordenación, mientras que las operaciones de unión y búsqueda cuestan aproximadamente $O(\alpha(n))$, esencialmente constante. Donde $\alpha$ hace referencia a la **función inversa de Ackermann**. 

>Nota:
>1. **Ackermann** es una función recursiva extremadamente creciente (más rápida que cualquier función exponencial o factorial).
>2. 2. **Su inversa**, $\alpha(n)$, crece tan lentamente que para cualquier valor práctico de `n` (incluso `n` del tamaño de átomos en el universo), $\alpha(n)\leq 4$ o $5$.  

Algunos puntos clave:

- Cuando decimos que cada operación de **find** o **union** cuesta $O(\alpha(n))$, estamos diciendo que, en el peor caso amortizado, su coste **crece aún más despacio** que cualquier logaritmo.  
- En la práctica, puedes considerarlo **"casi constante"**, puesto que $\alpha(n)$ apenas aumenta al crecer `n` por ejemplo:
  
  - Si $n=10^3$, $\alpha(n)=3$  
  - Si $n=10^8$, $\alpha(n)=4$  
  - Si $n\approx 10^{10}$, $\alpha(n)$ apenas llega a 5.  

Por eso, aunque teóricamente no sea **O(1)**, en la práctica el sobrecoste por cada llamada es insignificante.



**Uso en clustering aglomerativo jerárquico**

En **clustering aglomerativo**, deseamos fusionar iterativamente los clústeres más similares hasta formar una jerarquía:  

1. **Inicialización**: Cada punto de datos `x[i]` se añade a `UnionFindTrees` como clúster singleton.  
2. **Matriz de distancias**: Calcula la similitud o distancia entre todos los pares de puntos.  
3. **Selección de clústeres a fusionar**:  
   - Encuentra el par de clústeres `(C_a, C_b)` con la menor distancia según el criterio elegido (enlace simple, completo, promedio, Ward, etc.).  
   - Extrae un representante cualquiera de `C_a` y `C_b` (por ejemplo, un punto de cada conjunto). Use `find_partition` para identificar su raíz y comprobar si ya comparten clúster.  
   - Si son disjuntos, registra la fusión en el dendrograma y ejecuta `merge(elem_a, elem_b)`.  
4. **Actualización de distancias**:  
   - Después de fusionar, actualiza las distancias entre el nuevo clúster unido y el resto, según la estrategia de enlace.  
   - Repite la selección y fusión hasta que quede un solo clúster o se alcance el número deseado.  

El uso de **Union-Find** aquí evita reconstruir conjuntos desde cero: cada `merge` actualiza punteros en tiempo amortizado casi constante, mientras que `find_partition` identifica rápidamente a qué clúster pertenece cada punto. Esto permite llevar un registro eficiente de la partición de los datos en cada paso de la jerarquía, y facilita operaciones de consulta de pertenencia de puntos a clústeres sin tareas de búsqueda costosas en estructuras de datos explícitas.


### **Aplicaciones**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections.abc import Iterable as ABCIterable, Mapping
from typing import Dict, TypeVar, Generic, List, Tuple

T = TypeVar("T", bound=object)

class UnionFindError(TypeError):
    """Excepción personalizada para errores en UnionFind."""
    pass


def _err_ctor_illegal(val):
    return f"Argumento ilegal para el constructor: {val!r}"

def _err_ctor_duplicate(val):
    return f"Elemento duplicado en el conjunto inicial: {val!r}"

def _err_find_not_in_set(val):
    return f"El argumento {val!r} no está presente en la estructura UnionFind"

def _err_invalid_arg(method: str, param: str, val):
    return f"Argumento inválido en '{method}': parámetro '{param}' recibió valor {val!r}"

class UnionFindTrees(Generic[T]):
    """
    Estructura Disjoint-Set con compresión de ruta y unión por rango.
    """
    __slots__ = ("_parent", "_rank")

    def __init__(self, initial_set: ABCIterable[T] = None) -> None:
        if initial_set is None:
            initial_set = []
        if isinstance(initial_set, (str, bytes, Mapping)):
            raise UnionFindError(_err_ctor_illegal(initial_set))
        try:
            iterator = iter(initial_set)
        except TypeError:
            raise UnionFindError(_err_ctor_illegal(initial_set))

        self._parent: Dict[T, T] = {}
        self._rank: Dict[T, int] = {}
        for e in iterator:
            if e is None:
                raise UnionFindError(_err_ctor_illegal(e))
            if e in self._parent:
                raise UnionFindError(_err_ctor_duplicate(e))
            self._parent[e] = e
            self._rank[e] = 1

    def find_partition(self, elem: T) -> T:
        """
        Encuentra el representante (raíz) del conjunto al que pertenece elem,
        aplicando compresión de ruta.
        """
        if elem is None or elem not in self._parent:
            raise UnionFindError(_err_find_not_in_set(elem))
        root = elem
        while root != self._parent[root]:
            root = self._parent[root]
        while elem != root:
            parent = self._parent[elem]
            self._parent[elem] = root
            elem = parent
        return root

    def merge(self, a: T, b: T) -> bool:
        """
        Une los conjuntos que contienen a y b. Devuelve True si se unieron,
        False si ya estaban en el mismo conjunto.
        """
        r1 = self.find_partition(a)
        r2 = self.find_partition(b)
        if r1 == r2:
            return False
        if self._rank[r1] < self._rank[r2]:
            r1, r2 = r2, r1
        self._parent[r2] = r1
        self._rank[r1] += self._rank[r2]
        return True

class Graph(Generic[T]):
    """
    Grafo no dirigido con pesos opcionales en las aristas.
    """
    def __init__(self, vertices: ABCIterable[T] = None) -> None:
        self.adj: Dict[T, List[Tuple[T, float]]] = {}
        self.edges: List[Tuple[float, T, T]] = []
        if vertices:
            for v in vertices:
                self.add_vertex(v)

    def add_vertex(self, v: T) -> None:
        if v not in self.adj:
            self.adj[v] = []

    def add_edge(self, u: T, v: T, weight: float = 1.0) -> None:
        self.add_vertex(u)
        self.add_vertex(v)
        self.adj[u].append((v, weight))
        self.adj[v].append((u, weight))
        self.edges.append((weight, u, v))

    def vertices(self) -> List[T]:
        return list(self.adj.keys())

    def connected_components_dfs(self) -> List[List[T]]:
        visited = set()
        components: List[List[T]] = []
        def dfs(u: T, comp: List[T]):
            visited.add(u)
            comp.append(u)
            for v, _ in self.adj[u]:
                if v not in visited:
                    dfs(v, comp)
        for u in self.adj:
            if u not in visited:
                comp: List[T] = []
                dfs(u, comp)
                components.append(comp)
        return components

    def connected_components_unionfind(self) -> List[List[T]]:
        uf = UnionFindTrees(self.vertices())
        for _, u, v in self.edges:
            uf.merge(u, v)
        comps: Dict[T, List[T]] = {}
        for v in self.vertices():
            root = uf.find_partition(v)
            comps.setdefault(root, []).append(v)
        return list(comps.values())

    def kruskal_mst(self) -> List[Tuple[T, T, float]]:
        mst: List[Tuple[T, T, float]] = []
        uf = UnionFindTrees(self.vertices())
        for weight, u, v in sorted(self.edges, key=lambda x: x[0]):
            if uf.find_partition(u) != uf.find_partition(v):
                uf.merge(u, v)
                mst.append((u, v, weight))
        return mst

    def cluster_by_kruskal(self, k: int) -> List[List[T]]:
        uf = UnionFindTrees(self.vertices())
        edges_desc = sorted(self.edges, key=lambda x: x[0], reverse=True)
        removed = 0
        for weight, u, v in edges_desc:
            if uf.find_partition(u) != uf.find_partition(v):
                uf.merge(u, v)
                removed += 1
                if removed == len(self.vertices()) - k:
                    break
        comps: Dict[T, List[T]] = {}
        for v in self.vertices():
            root = uf.find_partition(v)
            comps.setdefault(root, []).append(v)
        return list(comps.values())

    def __repr__(self) -> str:
        return f"Graph(V={len(self.adj)}, E={len(self.edges)})"

#Funciones de visualización
def to_networkx(g: Graph[T]) -> nx.Graph:
    G = nx.Graph()
    G.add_nodes_from(g.vertices())
    for weight, u, v in g.edges:
        G.add_edge(u, v, weight=weight)
    return G

def draw_components(g: Graph[T], method: str = 'dfs') -> None:
    G = to_networkx(g)
    comps = (g.connected_components_unionfind() if method == 'unionfind'
             else g.connected_components_dfs())
    pos = nx.spring_layout(G, seed=42)
    plt.figure()
    for comp in comps:
        nx.draw_networkx_nodes(G, pos, nodelist=comp, node_size=300)
    nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_labels(G, pos)
    plt.title(f"Componentes conexas ({method})")
    plt.axis('off')
    plt.show()

def draw_mst(g: Graph[T]) -> None:
    G = to_networkx(g)
    mst_edges = [(u, v) for u, v, _ in g.kruskal_mst()]
    pos = nx.spring_layout(G, seed=42)
    plt.figure()
    nx.draw_networkx_nodes(G, pos, node_size=300)
    nx.draw_networkx_edges(G, pos, alpha=0.3)
    nx.draw_networkx_edges(G, pos, edgelist=mst_edges, width=2)
    nx.draw_networkx_labels(G, pos)
    plt.title("Árbol de recubrimiento mínimo")
    plt.axis('off')
    plt.show()

def draw_clusters(g: Graph[T], k: int) -> None:
    G = to_networkx(g)
    clusters = g.cluster_by_kruskal(k)
    pos = nx.spring_layout(G, seed=42)
    plt.figure()
    for cluster in clusters:
        nx.draw_networkx_nodes(G, pos, nodelist=cluster, node_size=300)
    nx.draw_networkx_edges(G, pos, alpha=0.2)
    nx.draw_networkx_labels(G, pos)
    plt.title(f"Clustering por Kruskal (k={k})")
    plt.axis('off')
    plt.show()

#Ejemplos de uso más complejos

# Ejemplo 1: Grafo numérico pequeño
print("\nEjemplo 1: Grafo numérico")
g1 = Graph([1, 2, 3, 4, 5, 6])
edges1 = [(1,2,1.0),(2,3,2.5),(4,5,1.2),(5,6,3.0),(3,4,2.0)]
for u,v,w in edges1:
    g1.add_edge(u, v, w)
print(g1)
print("Componentes (DFS):", g1.connected_components_dfs())
print("Componentes (UF):", g1.connected_components_unionfind())
print("MST:", g1.kruskal_mst())
print("Clusters k=2:", g1.cluster_by_kruskal(2))
draw_components(g1, method='dfs')
draw_mst(g1)

# Ejemplo 2: Grafo de cadenas y clustering k=3
print("\nEjemplo 2: Grafo con vértices tipo str")
vertices2 = ['A','B','C','D','E','F','G','H']
g2 = Graph(vertices2)
for (u,v),w in {('A','B'):2,('B','C'):1,('C','D'):3,('A','E'):2.5,('E','F'):0.5,('G','H'):4}.items():
    g2.add_edge(u, v, w)
print(g2)
print("Componentes (DFS):", g2.connected_components_dfs())
print("MST:", g2.kruskal_mst())
draw_clusters(g2, k=3)

# Ejemplo 3: Manejo de errores en UnionFind
print("\nEjemplo 3: Errores UnionFind")
uf = UnionFindTrees([1,2,3])
try:
    uf.find_partition(4)
except UnionFindError as err:
    print("Error detectado:", err)
# Unión válida vs redundante
print("Merge 1-2:", uf.merge(1,2))
print("Merge 1-2 de nuevo (debería ser False):", uf.merge(1,2))



### **Ejercicios** 

#### 1. Kruskal "industrial-grade" con *UnionFindTrees*  
Una empresa de telecomunicaciones va a desplegar fibra óptica en 20 000 ciudades conectadas por 300 000 posibles enlaces. Cada enlace tiene un coste de instalación y un "tiempo de vida" previsto. Necesitan un MST (Minimum Spanning Tree) que minimice coste y, a igualdad de coste, maximice longevidad. La decisión debe tomarse cada noche con los datos del día.  

**Tareas.**  

* Añade a `UnionFindTrees` un método `bulk_add(iterable)` que inserte masivamente nodos nuevos en **O(n)** amortizado sin invalidar rangos previos.  
* Implementa `kruskal_mst(edges, *, tie_break='max_life') -> list[Edge]`, donde los empates en coste se resuelven con un comparador secundario (p. ej. mayor longevidad).  
* Diseña pruebas unitarias parametrizadas (`pytest.mark.parametrize`) que verifiquen:  
  1. Correctitud del peso total.  
  2. Invariante de aciclicidad.  
  3. Uso estable del comparador secundario.  
* Crea un *benchmark* con `pytest-benchmark` que compare la versión clásica frente a tu versión con `bulk_add` en grafos Erdős-Rényi de 10 K-40 K vértices.  

#### 2. **k-Clustering** jerárquico con *UnionFindLists*  
En un sistema de recomendación de música se representan 150 000 canciones como puntos en $R^2$ (t-SNE). Quieren agruparlas con *single-linkage* hasta obtener exactamente *k* clústeres para experimentos A/B.  
**Tareas.**  

* Implementa `single_linkage(points, k)` reutilizando `UnionFindLists.merge`; detén el algoritmo cuando queden *k* particiones.  
* Añade un método `groups()` que devuelva las particiones ordenadas por tamaño descendente.  
* Integra un visualizador opcional que coloree cada clúster (usa *matplotlib*; **no** se evalúa en CI, pero documenta cómo activarlo).  
* Analiza teóricamente la complejidad O(min |A|, |B|) de cada fusión y discute cuándo *UnionFindTrees* sería preferible.  


#### 3. **MST dinámico** con operaciones *undo* (DSU-Rollback)  
 Para simulaciones de resiliencia de redes eléctricas se necesitan consultas *offline* del MST bajo inserción y eliminación temporal de aristas. Se conocen de antemano *q* consultas tipo "añadir arista" / "deshacer última"/"preguntar peso MST".  

**Tareas.**  

* Extiende `UnionFindTrees` con una pila de operaciones que guarde pares *(hijo, padre, rango_prev)*.  
* Implementa `rollback()` en **O(1)** que revierte la última `merge`.  
* Desarrolla `offline_mst(queries)` (divide-y-vencerás sobre el intervalo de consultas) y demuestra que responde en $O((n + m) \log n)$.  
* Redacta pruebas que cubran la secuencia de *rollback* hasta el estado inicial y verifiquen integridad de rangos.  


#### 4. **Borůvka paralelo** con *UnionFindTrees* thread-safe  
Un *render-farm* genera diariamente mallas 3D con hasta 50 M aristas. El pipeline se ejecuta en un clúster con 64 hilos y necesita el MST para simplificar mallas antes de la transmisión de vídeo.  

**Tareas.**  

* Haz `UnionFindTrees` seguro para concurrencia con *lock striping* (por bucket hash).  
* Implementa Borůvka multi-hilo: cada hilo busca la arista mínima saliente de su componente; sincroniza fusiones con CAS o `threading.Lock`.  
* Mide *speed-up* y *scaling efficiency* (8, 16, 32, 64 hilos) frente a Kruskal secuencial.  
* Entrega informe de 1 000 palabras sobre bottlenecks y afinamiento de granularidad de locks.  

#### 5. **Segmentación de imágenes** por componentes conexas (CCA)  
Para un proyecto de visión computacional se deben etiquetar todas las regiones blancas en un mapa binario de 4 096 × 4 096 píxeles.  
**Tareas.**  

* Escribe `label_components(bitmap)` que recorra la imagen en *raster-scan* y use `DisjointSet.merge` para unir píxeles vecinos 4-conexos.  
* Tras el primer pase, haz un segundo barrido que re-mapee cada componente a un entero consecutivo.  
* Calcula memoria máxima usada y demuestra que el algoritmo es $O(N \alpha(N))$.  

#### 6. **Algoritmo de *k*-clique-percolation** en grafos sociales  
Para analizar comunidades en un grafo de 2 M vértices se busca el método de *clique percolation*: dos k-cliques se unen si comparten k-1 vértices.  
**Tareas.**  

* Genera todas las 4-cliques con *pivot-branching*; asigna un id incremental a cada una.  
* Usa `UnionFindTrees.merge` para fusionar ids de cliques que percolan.  
* Devuelve el conteo y la distribución de tamaños de comunidades.  
* Optimiza para memoria restringida guardando solo ids de vértices ordenados y usando hashing incremental.  


#### 7. **Detección de *percolación* en redes hexagonales**  
En física estadística se estudia si existe un camino de sitios ocupados que conecte de arriba hacia abajo en una malla hexagonal $L\times L$.  
**Tareas.**  

* Modela la malla como un conjunto de sitios con índices (q, r).  
* Inserta un sitio ocupado │-> `add` + conexiones a sus seis vecinos.  
* Tras cada inserción responde en $O(\alpha(N))$ si "percola" (las particiones de la fila superior e inferior comparten raíz).  
* Simula 10 000 corridas Monte-Carlo y estima $p_n^{*}$ (probabilidad crítica) para L = 128.  


#### 8. **Persistencia distribuida de particiones** con *hash-ring*  
Un sistema de *microservicios* mantiene su estado de conectividad en 256 shards. Cada shard aloja un sub-DSU que debe sincronizarse cada minuto con sus vecinos inmediatos en el hash-ring.  
**Tareas.**  

* Serializa cada sub-DSU (`pickle` o JSON propio) y publícalo en Kafka.  
* Implementa reconciliación incremental: solo se envían pares *(elem, nueva_raíz)* si la raíz cambió.  
* Exhibe pruebas de consistencia eventual en presencia de particiones de red (usa `tox` + Docker Compose).  

**Recomendaciones de entrega**

* **Estructura** tu repositorio con `src/`, `tests/`, `benchmarks/` y `docs/`.  
* Configura **CI/CD** en GitHub Actions que ejecute pytest, linters y, si procede, *stress tests* con límites de tiempo.  
* Incluye *type hints* (`mypy --strict`) y asegura $\geq$ 95 % de cobertura.  
* Documenta en un `README.md` cada ejercicio con instrucciones para reproducir resultados y scripts de generación de datos.

Presenta tus respuestas en tu repositorio